In [7]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [8]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'Factory+Automation' # topic is "large language model"
dataset = '&sp=3178' # dataset: Tech blogs
rows = str(10) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/lutzenrath/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']#[:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(current_doc['content'])        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


# Answering questions on the results with OpenAI GPT

In [9]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 250 words.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


system_prompt = f"""
You are a partner at a large venture capital firm who has decades of experience in the founding and growing of successful companies. You are currently helping a large corporate partner to find areas for innovation. For this, you utilise the mom-test.

The mom-test is a strategy for getting insights out of potential users, even when said potential users may be prone to lying to you. You mom is the example in this strategy: she will say she likes something because it’s you that’s asking.

You don’t ask your mom whether she likes your idea for a new cookbook app. You ask her how, when, and why she uses her device, and about her current cookbooks. The question about your idea will likely lead to a false answer, while the questions about her activities (a.k.a Mom Test questions) will give you insight into what she actually likes and wants from a product.

In essence, you’re getting to the bottom of what the user actually wants, rather than trying to force your idea down their throat. You’re listening to the user without misinterpreting what they’re saying.

When a customer makes a request, get them to explain exactly how and why that would help them.


"""


user_prompt = f"""
Consider the text below, delimited by ```, in the area of {query}:
```
{summary_list}
```
Please identify problems that are experienced by customers in the above text and answer the following mom-test questions from the perspective of one of these customers:

- Why do you want that?
- What would that empower you to do?
- How are you managing without it?
- How would that fit into your day?
- Do you think we should push back the launch to add the feature, or is it something we could include at a later date?
- See how these questions lead the user to give more detailed feedback, which in turn creates more powerful research data for you?

Please provide some sources of inspiration that you used from the document excerpts
"""
 
completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

print(completion.choices[0].message)  
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability 

Redflow, an Australian energy storage company, is developing a new zinc bromide flow battery, the "X10," to compete with large-scale battery technologies like lithium. The company is considering onshoring manufacturing to Australia and potentially opening a factory in the US to comply with clean energy legislation. Despite expected financial losses, Redflow has secured contracts in the US market and aims to supply significant battery storage capacities to projects like microgrids and hospitals. The company is focused on long-term success, addressing challenges in production and aiming to improve automation to reduce costs. Redflow's CEO emphasizes the company's advantages in environmental impact, safety, and deployment costs compared to competitors. The company is planning to increase production capacity in Thailand and introduce automation to potentially double production levels to meet increasing demand for energy storage solutions. Overall, Redflow is strategically positioned to com